# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-90fb9589ee-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

## Task 1
Identify the BGP for Cultural Movement

In [37]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P1138'), ('name', 'Kunstindeks Danmark Artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1248'), ('name', 'KulturNav-ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), (

In [44]:
#[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 wdt:P135 ?p .
   # get the label
   ?p sc:name ?name.
   
   
  # filter(regex(?name,"cultural movement"))
  #propertysysubjectkesenikalen
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
1


Final query for this task

In [73]:
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q1472236 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P156')]
[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P361')]
[('p', 'http://www.wikidata.org/entity/Q968159'), ('name', 'art movement'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q979160'), ('name', 'Proto-Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P155')]
5


In [74]:
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'),


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q4692 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13830551'), ('name', 'Portal:Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P1151')]
[('p', 'http://www.wikidata.org/entity/Q15885248'), ('name', 'Renaissance painting'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1781608'), ('name', 'Renaissance literature'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1404472'), ('name', 'Italian Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný'), ('o', 'http://www.wikidata.org/prop/direct/P1343')]
[('p', 'http://www.wikidata.org/entity/Q2749

In [75]:
#final query task 1 BGP OF CULTURAL MOVEMENT.
#[('p', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'),


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q2198855 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/P135'), ('name', 'movement'), ('o', 'http://www.wikidata.org/prop/direct/P1687')]
[('p', 'http://www.wikidata.org/entity/Q49773'), ('name', 'social movement'), ('o', 'http://www.wikidata.org/prop/direct/P279')]
[('p', 'http://www.wikidata.org/entity/Q3533467'), ('name', 'group action'), ('o', 'http://www.wikidata.org/prop/direct/P279')]
[('p', 'http://www.wikidata.org/entity/Q6638204'), ('name', 'Category:Cultural movement'), ('o', 'http://www.wikidata.org/prop/direct/P910')]
4


## Task 2
Identify the BGP for Renassaince, Baroque and Romanticism

In [38]:
# FOR RENASSAINCE
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1472236 ?p ?o .
   # get the label
   ?p sc:name ?name.
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P5604'), ('name', 'archINFORM keyword ID')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('name', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', '

In [47]:
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q1472236 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P156')]
[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P361')]
[('p', 'http://www.wikidata.org/entity/Q968159'), ('name', 'art movement'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q979160'), ('name', 'Proto-Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P155')]
5


Final query for this task

In [48]:
# BGP OF RENAISSANCE
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'),


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q4692 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q13830551'), ('name', 'Portal:Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P1151')]
[('p', 'http://www.wikidata.org/entity/Q15885248'), ('name', 'Renaissance painting'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1781608'), ('name', 'Renaissance literature'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1404472'), ('name', 'Italian Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný'), ('o', 'http://www.wikidata.org/prop/direct/P1343')]
[('p', 'http://www.wikidata.org/entity/Q2749

In [92]:
# BGP OF BAROQUE
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque'),


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   wd:Q37853 ?o ?p .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q172991'), ('name', 'Council of Trent'), ('o', 'http://www.wikidata.org/prop/direct/P737')]
[('p', 'http://www.wikidata.org/entity/Q11514315'), ('name', 'historical period'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q1292119'), ('name', 'style'), ('o', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q131808'), ('name', 'Mannerism'), ('o', 'http://www.wikidata.org/prop/direct/P155')]
[('p', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný'), ('o', 'http://www.wikidata.org/prop/direct/P1343')]
[('p', 'http://www.wikidata.org/entity/Q2657718'), ('name', 'Armenian Soviet Encyclopedia'), ('o', 'http://www.wikidata.org/prop/direct/P1343')]
[('p', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia'), ('o', 'http://www.wikidata.org/prop/direct/P1343')]
[('p', 'http://www.wikidata.org/entity/Q32880'), ('name', 'ar

In [79]:
#exploring cultural movement fpr romanticism
#[('p', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm
WHERE {
   # bind something
   ?p wdt:P31 wd:Q2198855 .
   # get the label
   ?p sc:name ?name.
   
   

   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q8210150'), ('name', 'Avanzada Regia')]
[('p', 'http://www.wikidata.org/entity/Q211481'), ('name', 'Dogme 95')]
[('p', 'http://www.wikidata.org/entity/Q2046861'), ('name', 'Palaeologan Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1400430'), ('name', 'Macedonian Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q430167'), ('name', 'Spanish Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1814286'), ('name', 'Young Estonia')]
[('p', 'http://www.wikidata.org/entity/Q2614483'), ('name', 'Pre-romanticism')]
[('p', 'http://www.wikidata.org/entity/Q39427'), ('name', 'surrealism')]
[('p', 'http://www.wikidata.org/entity/Q2349908'), ('name', 'French Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q878985'), ('name', 'modernism')]
[('p', 'http://www.wikidata.org/entity/Q466887'), ('name', 'Belle Époque')]
[('p', 'http://www.wikidata.org/entity/Q12539'), ('name', 'Age of Enlightenment')]
[('p', 'http://www.wikidata.org/entit

In [91]:
# BGP OF BAROQUE AND RENAISSANCE AND ROMANTICISM
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance'),
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]


queryString = """
SELECT DISTINCT ?p ?name ?o ?nm ?p1 ?name1
WHERE{ {
   # bind something
    wd:Q37853 ?p ?o.
   # get the label
   ?p sc:name ?name.   
}
Union
{
# bind something
    wd:Q4692 ?p1 ?o.
   # get the label
   ?p1 sc:name ?name1.
   }
   Union
{
# bind something
    wd:Q37068 ?x ?o.
   # get the label
   ?x sc:name ?nm.
   }
   }
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID'), ('o', '300021147')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID'), ('o', '11644')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning'), ('o', '687754')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID'), ('o', '0162729')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID'), ('o', '0007955')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID'), ('o', '0083458')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2041543')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 

## Task 3
Identify the BGP for composer, painter, sculptor and poet

In [81]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('p', 'http://www.wikidata.org/entity/Q104694359'), ('name', 'WikiProject PCC Wikidata Pilot/Frick Art Reference Library')]
[('p', 'http://www.wikidata.org/entity/Q106675203'), ('name', 'WikiProject PCC Wikidata Pilot/Berenson Library')]
[('p', 'http://www.wikidata.org/entity/Q1081881'), ('name', 'National Museum of San Matteo')]
[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
[('p', 'http://www.wikidata.org/entity/Q13362'), ('name', 'Ferrara')]
[('p', 'http://www.wikidata.org/entity/Q13375'), ('name', 'Pisa')]
[('p', 'http://www.wikidata.org/entity/Q134307'), ('name', 'portrait')]
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q148540'), ('name', 'Republic of Florence')]
[('p', 'http://www.wikidata.org/entity/Q180788'), ('name', 'National Gallery')]
[('p', 'http://www.wikidata.org/entity/Q2044'), ('name', 'F

In [82]:
#Including the 2 Numbers from last result (Painter and Scupltor)
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1028181 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1027879'), ('name', 'graphics')]
[('p', 'http://www.wikidata.org/entity/Q128538'), ('name', 'Luke the Evangelist')]
[('p', 'http://www.wikidata.org/entity/Q5914426'), ('name', 'Template:Infobox artist')]
[('p', 'http://www.wikidata.org/entity/Q6615720'), ('name', 'Category:Painters')]
[('p', 'http://www.wikidata.org/entity/Q11629'), ('name', 'art of painting')]
[('p', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/entity/Q174705'), ('name', 'oil painting')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q174219'), ('name', 'paint')]
[('p', 'http://www.wikidata.org/entity/Q189085'), ('name', 'pastel')]
[('p', 'http://www.wikidata.org/entity/Q3391743'), ('name', 'visual artist')]
[('p', 'http://www.wikidata.org/entity/Q193897'), ('name', 'paintbrush')]
[('p', 'http://www.wikidata.org/entity/Q3300034'), ('name', 'painting ma

In [83]:
#exploring occupation for composer and poet
#[('p', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q12737077 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q4048530'), ('name', 'Category:Occupations')]
[('p', 'http://www.wikidata.org/entity/Q151885'), ('name', 'concept')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q4164871'), ('name', 'position')]
[('p', 'http://www.wikidata.org/entity/Q24017414'), ('name', 'second-order class')]
[('p', 'http://www.wikidata.org/entity/Q1807498'), ('name', 'social position')]
[('p', 'http://www.wikidata.org/entity/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/entity/P3095'), ('name', 'practiced by')]
[('p', 'http://www.wikidata.org/entity/P425'), ('name', 'field of this occupation')]
9


In [87]:
#exploring occupation for composer and poet
#[('p', 'http://www.wikidata.org/entity/P106'), ('name', 'occupation')]queryString = """
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   filter(regex(?name,"Composer"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q15252222'), ('name', 'Composer')]
[('p', 'http://www.wikidata.org/entity/Q16387857'), ('name', 'Composers Union of Armenia')]
[('p', 'http://www.wikidata.org/entity/Q8008937'), ('name', 'Category:Composers')]
[('p', 'http://www.wikidata.org/entity/Q1754714'), ('name', 'Media Composer')]
[('p', 'http://www.wikidata.org/entity/Q1121767'), ('name', 'Composer')]
5


In [89]:
#exploring occupation for composer and poet
#[('p', 'http://www.wikidata.org/entity/P106'), ('name', 'occupation')]queryString = """
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   filter(regex(?name,"poet"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q106376813'), ('name', 'visual poet')]
[('p', 'http://www.wikidata.org/entity/Q1209498'), ('name', 'poet lawyer')]
[('p', 'http://www.wikidata.org/entity/Q482'), ('name', 'poetry')]
[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('p', 'http://www.wikidata.org/entity/Q182357'), ('name', 'lyric poetry')]
[('p', 'http://www.wikidata.org/entity/Q11386386'), ('name', 'haiku poet')]
[('p', 'http://www.wikidata.org/entity/Q11122954'), ('name', 'tanka poet')]
[('p', 'http://www.wikidata.org/entity/Q23117687'), ('name', 'dithyrambic poet')]
[('p', 'http://www.wikidata.org/entity/Q1237928'), ('name', 'poet laureate')]
[('p', 'http://www.wikidata.org/entity/Q2293636'), ('name', 'slam poet')]
[('p', 'http://www.wikidata.org/entity/Q7168247'), ('name', 'performance poetry')]
[('p', 'http://www.wikidata.org/entity/Q7207537'), ('name', 'poetry reading')]
[('p', 'http://www.wikidata.org/entity/Q107714046'), ('name', 'performance poet'

Final query for this task

In [90]:
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
#[('p', 'http://www.wikidata.org/entity/Q1121767'), ('name', 'Composer')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
queryString = """
SELECT DISTINCT ?p ?name ?o ?nm ?p1 ?name1 ?nmm ?u
WHERE{ {
   # bind something
    wd:Q49757 ?p ?o.
   # get the label
   ?p sc:name ?name.   
}
Union
{
# bind something
    wd:Q1121767 ?p1 ?o.
   # get the label
   ?p1 sc:name ?name1.
   }
   Union
{
# bind something
    wd:Q1028181 ?x ?o.
   # get the label
   ?x sc:name ?nm.
   }
   
   Union
{
# bind something
    wd:Q1281618 ?u ?z.
   # get the label
   ?u sc:name ?nmm.
   }
   }
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID'), ('o', '300025513')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID'), ('o', '0133026')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class'), ('o', 'http://dbpedia.org/ontology/Poet')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('o', 'http://www.wikidata.org/entity/Q24054302')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P5509')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P5477')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'

## Task 4
Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query

## Task 5
Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query

## Task 6
For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query

## Task 7
Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query